In [1]:
from sqlalchemy import create_engine
from pandas.io.sql import read_sql

conn_string = 'postgresql://iangow.me/crsp'
engine = create_engine(conn_string)

In [2]:
tone_data = read_sql("SELECT category, word_list FROM lm_tone", engine)
tone_data = tone_data.set_index(['category'])

In [3]:
import re

def make_regex(word_list):
    mod_word_list = [word.lower() for word in word_list]
    regex = '\\b(?:' + '|'.join(mod_word_list) + ')\\b'
    return  re.compile(regex)

tone_data
categories = [cat for cat in tone_data.index]

In [4]:
def get_regex_matches(category, text):
    regex = make_regex(tone_data.ix[category]['word_list'])
    return re.findall(regex, text)

In [5]:
file_name = '999419_T'

sql = """
    SELECT speaker_name, role, context,
        file_name, last_update, speaker_number, speaker_text
    FROM streetevents.speaker_data
    WHERE file_name='%s'
    ORDER BY context, speaker_number
""" % (file_name)

df = read_sql(sql, engine)
some_text = df["speaker_text"][3]
some_text

"Thank you, Mike. We continue to see major success with the Company's transition to high, value-added products and increased profitability. IR proprietary product revenues grew 39 percent year-on-year and now comprise 69 percent of Company revenues. The fastest growing segment of our business, analog ICs and advance circuit devices, grew 47 percent year-on-year and now comprise 57 percent of our revenues. In fact, sales from these products have more than doubled in the last eight quarters fueled by success in both IT and energy-conserving applications. This richer mix has enabled us to grow our gross margins 11 percentage points, setting a new high of 43.3 percent in this quarter. In the December quarter, we saw double-digit revenue growth across all end-markets, year-over-year. Also, we realized record proprietary product design wins, which grew 10 percent, sequentially, and 33 percent year-on-year. In Information Technology, we grew revenues 30 percent year-over-year, and we continue

In [6]:
def get_tone_data(text):
    text = text.lower()
    the_dict = {category: len(get_regex_matches(category, text))
                            for category in categories}
    return the_dict

In [7]:
get_tone_data(some_text)

{'litigious': 0,
 'modal_strong': 2,
 'modal_weak': 1,
 'negative': 3,
 'positive': 15,
 'uncertainty': 1}

In [8]:
df["tone_count"] = df["speaker_text"].map(get_tone_data)
df.drop(["speaker_name", "role", "speaker_text"], axis=1, inplace=True)

In [9]:
for key in categories:
    df[key] = df["tone_count"].map(lambda x: x[key])
df.drop(["tone_count"], axis=1, inplace=True)

In [10]:
df

,context,file_name,last_update,speaker_number,positive,negative,litigious,modal_strong,modal_weak,uncertainty
0,pres,999419_T,2005-01-28 14:26:07,1,0,0,0,0,0,0
1,pres,999419_T,2005-01-28 14:26:07,2,1,3,1,2,2,4
2,pres,999419_T,2005-01-28 14:26:07,3,2,7,2,1,0,4
3,pres,999419_T,2005-01-28 14:26:07,4,15,3,0,2,1,1
4,pres,999419_T,2005-01-28 14:26:07,5,1,5,0,0,0,0
5,pres,999419_T,2005-01-28 14:26:07,6,0,1,0,0,0,0
6,qa,999419_T,2005-01-28 14:26:07,1,0,0,0,1,0,1
7,qa,999419_T,2005-01-28 14:26:07,2,0,3,0,2,0,0
8,qa,999419_T,2005-01-28 14:26:07,3,0,2,0,0,0,0
9,qa,999419_T,2005-01-28 14:26:07,4,1,0,0,0,0,0


In [11]:
# Next write data to PostgreSQL